In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error


In [2]:
train = pd.read_csv("./Dataset/train.csv", parse_dates=['date'])
stores = pd.read_csv('./Dataset/stores.csv')
transactions = pd.read_csv('./Dataset/transactions.csv', parse_dates=['date'])
oil = pd.read_csv('./Dataset/oil.csv', parse_dates=['date'])
events = pd.read_csv('./Dataset/holidays_events.csv', parse_dates=['date'])
test = pd.read_csv('Dataset/test.csv', parse_dates=['date'])

transfer_dates = events.loc[events['type'] == 'Transfer', 'date']
holiday_dates  = events.loc[
    (events['type'] == 'Holiday') & (~events['transferred']),
    'date'
]
real_holidays = pd.concat([transfer_dates, holiday_dates]).drop_duplicates()


In [3]:
def merge_external(df):
    df = df.merge(stores, on='store_nbr', how='left')
    df = df.merge(transactions, on=['date', 'store_nbr'], how='left')
    df = df.merge(oil, on='date', how='left')
    df = df.merge(events[['date','type','transferred']], on='date', how='left')
    return df

train_merged = merge_external(train)
test_merged  = merge_external(test)

'''train = merge_external(train)
test  = merge_external(test)

train.to_csv("./Dataset/train_merged.csv", index=False)
test.to_csv("./Dataset/test_merged.csv", index=False)'''



'train = merge_external(train)\ntest  = merge_external(test)\n\ntrain.to_csv("./Dataset/train_merged.csv", index=False)\ntest.to_csv("./Dataset/test_merged.csv", index=False)'

## Feature engineering

In [4]:
def create_features(df):
    df['dayofweek'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['weekends'] = df['dayofweek'].isin([5,6]).astype(int)

    df['promo']  = df['onpromotion'].astype(int)
    df['family']= df['family'].astype('category').cat.codes

    df['is_holiday'] = df['date'].isin(real_holidays).astype(int)
    df['is_bridge_day'] = (df['type_y'] == 'Bridge').astype(int)
    df['is_work_day'] = (df['type_y'] == 'Work Day').astype(int)
    df['is_additional_holiday'] = (df['type_y'] == 'Additional').astype(int)

    df['day'] = df['date'].dt.day
    df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
    df['is_payday'] = ((df['day'] == 15) | (df['is_month_end'] == 1)).astype(int)

    quake_start = pd.to_datetime('2014-06-16')
    quake_end = quake_start + pd.Timedelta(days=30)
    df['post_earthquake'] = ((df['date'] >= quake_start) & (df['date'] <= quake_end)).astype(int)

    df['transactions'] = df['transactions'].fillna(0)
    df['oil_missing'] = df['dcoilwtico'].isna().astype(int)
    df['is_holiday'] = df['date'].isin(real_holidays).astype(int)
    df['dcoilwtico'] = df['dcoilwtico'].ffill().bfill()


    return df

train_merged = create_features(train_merged)
test_merged = create_features(test_merged)

    

## EDA

In [5]:
train_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 28 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   id                     int64         
 1   date                   datetime64[ns]
 2   store_nbr              int64         
 3   family                 int8          
 4   sales                  float64       
 5   onpromotion            int64         
 6   city                   object        
 7   state                  object        
 8   type_x                 object        
 9   cluster                int64         
 10  transactions           float64       
 11  dcoilwtico             float64       
 12  type_y                 object        
 13  transferred            object        
 14  dayofweek              int32         
 15  month                  int32         
 16  year                   int32         
 17  weekends               int64         
 18  promo                 

In [6]:
train_merged.shape

(3054348, 28)

In [7]:
train_merged.head(50)

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,promo,is_holiday,is_bridge_day,is_work_day,is_additional_holiday,day,is_month_end,is_payday,post_earthquake,oil_missing
0,0,2013-01-01,1,0,0.0,0,Quito,Pichincha,D,13,...,0,1,0,0,0,1,0,0,0,1
1,1,2013-01-01,1,1,0.0,0,Quito,Pichincha,D,13,...,0,1,0,0,0,1,0,0,0,1
2,2,2013-01-01,1,2,0.0,0,Quito,Pichincha,D,13,...,0,1,0,0,0,1,0,0,0,1
3,3,2013-01-01,1,3,0.0,0,Quito,Pichincha,D,13,...,0,1,0,0,0,1,0,0,0,1
4,4,2013-01-01,1,4,0.0,0,Quito,Pichincha,D,13,...,0,1,0,0,0,1,0,0,0,1
5,5,2013-01-01,1,5,0.0,0,Quito,Pichincha,D,13,...,0,1,0,0,0,1,0,0,0,1
6,6,2013-01-01,1,6,0.0,0,Quito,Pichincha,D,13,...,0,1,0,0,0,1,0,0,0,1
7,7,2013-01-01,1,7,0.0,0,Quito,Pichincha,D,13,...,0,1,0,0,0,1,0,0,0,1
8,8,2013-01-01,1,8,0.0,0,Quito,Pichincha,D,13,...,0,1,0,0,0,1,0,0,0,1
9,9,2013-01-01,1,9,0.0,0,Quito,Pichincha,D,13,...,0,1,0,0,0,1,0,0,0,1


In [8]:
train_merged.iloc[:,3:].describe()

,family,sales,onpromotion,cluster,transactions,dcoilwtico,dayofweek,month,year,weekends,promo,is_holiday,is_bridge_day,is_work_day,is_additional_holiday,day,is_month_end,is_payday,post_earthquake,oil_missing
count,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06,3.054348e+06
mean,1.600000e+01,3.590209e+02,2.617480e+00,8.481481e+00,1.558656e+03,6.785992e+01,3.008168e+00,6.222870e+00,2.014842e+03,2.876313e-01,2.617480e+00,1.061844e-01,1.750292e-03,2.917153e-03,2.217036e-02,1.565694e+01,3.208868e-02,6.476079e-02,2.042007e-02,3.127188e-01
std,9.521906e+00,1.107286e+03,1.225494e+01,4.649735e+00,1.036468e+03,2.566330e+01,2.001442e+00,3.374251e+00,1.345607e+00,4.526584e-01,1.225494e+01,3.080735e-01,4.179987e-02,5.393185e-02,1.472374e-01,8.801398e+00,1.762357e-01,2.461033e-01,1.414323e-01,4.636009e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.619000e+01,0.000000e+00,1.000000e+00,2.013000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,9.310000e+02,4.632000e+01,1.000000e+00,3.000000e+00,2.014000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.600000e+01,1.100000e+01,0.000000e+00,8.500000e+00,1.332000e+03,5.341000e+01,3.000000e+00,6.000000e+00,2.015000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.600000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.400000e+01,1.960110e+02,0.000000e+00,1.300000e+01,1.980000e+03,9.572000e+01,5.000000e+00,9.000000e+00,2.016000e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,3.200000e+01,1.247170e+05,7.410000e+02,1.700000e+01,8.359000e+03,1.106200e+02,6.000000e+00,1.200000e+01,2.017000e+03,1.000000e+00,7.410000e+02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.100000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [ ]:
train_merged['dcoilwtico'].mean()

np.float64(67.8599241540257)

## Modelling

In [ ]:
features = [
    'store_nbr','family','dayofweek','month','year','weekend',
    'promo','transactions','dcoilwtico',
    'is_holiday','is_bridge_day','is_work_day','is_additional_holiday',
    'is_payday','post_earthquake'
]
X = train_merged[features]
y = train_merged['sales']
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)
model = RandomForestRegressor(
    n_estimators=100, n_jobs=-1, random_state=42
)
model.fit(X_train, y_train)

val_preds = model.predict(X_val)
rmsle = np.sqrt(
    mean_squared_log_error(y_val.clip(0), val_preds.clip(0))
)
print(f'Validation RMSLE: {rmsle:.4f}')


Validation RMSLE: 0.4080
